<a href="https://colab.research.google.com/github/KaveeshBaddage/DataScienceImpl/blob/main/Copy_of_Generative_Adversarial_Network_(GAN)_for_MNIST_kaveesha_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

import pickle as pkl

from google.colab import files

import numpy as np

#to download the zipped files
import shutil

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Load MNIST Data set to Run Time



transform=ToTensor() <- converts data in the range 0-255 to 0-1.



In [8]:
transform = transforms.Compose([transforms.ToTensor(),
  transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(
    root='data/',
    train=True,
    download=True,
    transform=transform
    )  
print("train_dataset.data.size() ->",train_dataset.data.size())

# Dataset stores the samples and their corresponding labels, and DataLoader wraps an iterable around the Dataset to enable easy access to the samples.
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, 
    batch_size=100,
    shuffle=True
    # num_workers=0
    )

test_dataset = datasets.MNIST(
    root='data/', 
    train=False, 
    transform=transform
    )
print("test_dataset_sizec ->",test_dataset.data.size())


test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, 
    batch_size=100,
    # num_workers=0
    shuffle=True
    )

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw

train_dataset.data.size() -> torch.Size([60000, 28, 28])
test_dataset_sizec -> torch.Size([10000, 28, 28])


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


# torch.utils.data.DataLoader Parameters

dataset (Dataset) – dataset from which to load the data.

batch_size (int, optional) – how many samples per batch to load (default: 1).

num_workers (int, optional) – how many subprocesses to use for data loading. 0 means that the data will be loaded in the main process.


----------

The iter() function creates an object which can be iterated one element at a time.



In [9]:
# defining Discriminator class

class Discriminator(nn.Module):

    def __init__(self, input_size, d_output_size):
        # access methods from parent class nn.Module
        super(Discriminator, self).__init__()
        
        # define hidden linear layers
        self.fc1 = nn.Linear(input_size, 1024)
        self.fc2 = nn.Linear(self.fc1.out_features, self.fc1.out_features//2)
        self.fc3 = nn.Linear(self.fc2.out_features, self.fc1.out_features//2)
        
        # final fully-connected layer
        self.fc4 = nn.Linear(self.fc3.out_features, d_output_size)
        
    def forward(self, x):
        # flatten image
        x = x.view(-1, 28*28) #  reshape the tensor to 28*28 tensor
        # all hidden layers
        x = F.leaky_relu(self.fc1(x), 0.2) # (input, negative_slope=0.2) This function returns x if it receives any positive input, but for any negative value of x, it returns a really small value which is 0.02 times x
        x = F.dropout(x, 0.3)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.dropout(x, 0.3)
        x = F.leaky_relu(self.fc3(x), 0.2)
        x = F.dropout(x, 0.3)
        # final layer
        out = torch.sigmoid(self.fc4(x))


        return out

In [10]:
# defining generator class 1

class Generator(nn.Module):

    def __init__(self, input_size, g_output_size):
        super(Generator, self).__init__()
        
        # define hidden linear layers
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(self.fc1.out_features, self.fc1.out_features*2)
        self.fc3 = nn.Linear(self.fc2.out_features, self.fc1.out_features*2)
        
        # final fully-connected layer
        self.fc4 = nn.Linear(self.fc3.out_features, g_output_size)

    def forward(self, x):
        # all hidden layers
        x = F.leaky_relu(self.fc1(x), 0.2) # (input, negative_slope=0.2)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.leaky_relu(self.fc3(x), 0.2)
        out = torch.tanh(self.fc4(x))

        return out

Reference - https://deeplizard.com/learn/video/bH9Nkg7G8S0

nn.Sequential - A sequential container. Modules will be added to it in the order they are passed in the constructor
The Sequential class allows us to build PyTorch neural networks on-the-fly without having to build an explicit class

---------

Rectified Linear Activation Function(ReLU) do

if(input > 0){
	return input 
  }
else{
	return 0
}

In [11]:

# Discriminator hyperparams

# Size of input image to discriminator (28*28)
input_size = 784
# Size of discriminator output (real or fake)
d_output_size = 1
# Size of last hidden layer in the discriminator
d_hidden_size = 64

# Generator hyperparams

# Size of latent vector to give to generator
z_size = 100
# Size of discriminator output (generated image)
g_output_size = 784
# Size of first hidden layer in the generator
g_hidden_size = 32


D = Discriminator(input_size, d_output_size).to(device)
G = Generator(z_size, g_output_size).to(device)

# check that they are as you expect
print(D)
print()
print(G)

Discriminator(
  (fc1): Linear(in_features=784, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=1, bias=True)
)

Generator(
  (fc1): Linear(in_features=100, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=784, bias=True)
)


In [6]:
# loss
criterion = nn.BCELoss() 

# optimizer
lr = 0.0002 
G_optimizer = optim.Adam(G.parameters(), lr = lr)
D_optimizer = optim.Adam(D.parameters(), lr = lr)

In [7]:
def D_train(x):
    #=======================Train the discriminator=======================#
    D.zero_grad()

    # train discriminator on real
    x_real, y_real = x.view(-1, 28*28), torch.ones(100, 1)
    x_real, y_real = Variable(x_real.to(device)), Variable(y_real.to(device))

    D_output = D(x_real)
    D_real_loss = criterion(D_output, y_real)
    D_real_score = D_output

    # train discriminator on facke
    z = Variable(torch.randn(100, 100).to(device))
    x_fake, y_fake = G(z), Variable(torch.zeros(100, 1).to(device))

    D_output = D(x_fake)
    D_fake_loss = criterion(D_output, y_fake)
    D_fake_score = D_output

    # gradient backprop & optimize ONLY D's parameters
    D_loss = D_real_loss + D_fake_loss
    D_loss.backward()
    D_optimizer.step()
        
    return  D_loss.data.item()

In [8]:
def G_train(x):
    #=======================Train the generator=======================#
    G.zero_grad()

    z = Variable(torch.randn(100, 100).to(device))
    y = Variable(torch.ones(100, 1).to(device))

    G_output = G(z)
    D_output = D(G_output)
    G_loss = criterion(D_output, y)

    # gradient backprop & optimize ONLY G's parameters
    G_loss.backward()
    G_optimizer.step()
        
    return G_loss.data.item()

*    train_loader contains 938 batches and each batch has 64 tensors. Because we define batch_size as 64 when storing dataset in to the DataLoader.
    *    938 X 64 = 60032  // which equals to train_dataset:60000 
*    Each tensor in a batch represent single image. 
*    Image representing tensor  has following format.
    *    `[[28 real numbers][28 real numbers]]`



In [9]:
n_epoch = 200
for epoch in range(1, n_epoch+1):           
    D_losses, G_losses = [], []
    for batch_idx, (x, _) in enumerate(train_loader):
        D_losses.append(D_train(x))
        G_losses.append(G_train(x))

    print('[%d/%d]: loss_d: %.3f, loss_g: %.3f' % (
            (epoch), n_epoch, torch.mean(torch.FloatTensor(D_losses)), torch.mean(torch.FloatTensor(G_losses))))

[1/200]: loss_d: 0.748, loss_g: 3.923
[2/200]: loss_d: 0.782, loss_g: 2.603
[3/200]: loss_d: 0.791, loss_g: 2.585
[4/200]: loss_d: 0.647, loss_g: 2.833
[5/200]: loss_d: 0.356, loss_g: 3.651
[6/200]: loss_d: 0.416, loss_g: 3.748
[7/200]: loss_d: 0.448, loss_g: 3.351
[8/200]: loss_d: 0.482, loss_g: 3.035
[9/200]: loss_d: 0.490, loss_g: 3.161
[10/200]: loss_d: 0.520, loss_g: 2.988
[11/200]: loss_d: 0.540, loss_g: 2.736
[12/200]: loss_d: 0.552, loss_g: 2.706
[13/200]: loss_d: 0.600, loss_g: 2.437
[14/200]: loss_d: 0.652, loss_g: 2.336
[15/200]: loss_d: 0.651, loss_g: 2.338
[16/200]: loss_d: 0.730, loss_g: 2.283
[17/200]: loss_d: 0.698, loss_g: 2.214
[18/200]: loss_d: 0.738, loss_g: 2.095
[19/200]: loss_d: 0.808, loss_g: 1.891
[20/200]: loss_d: 0.792, loss_g: 2.002
[21/200]: loss_d: 0.792, loss_g: 1.947
[22/200]: loss_d: 0.792, loss_g: 1.921
[23/200]: loss_d: 0.862, loss_g: 1.737
[24/200]: loss_d: 0.872, loss_g: 1.721
[25/200]: loss_d: 0.891, loss_g: 1.667
[26/200]: loss_d: 0.913, loss_g: 1

Enumerate() method adds a counter to an iterable and returns it in a form of enumerate objec

In [16]:

# Save the Generator
with open('G.pkl', 'wb') as f:
    pkl.dump(G, f)

# Save the Discriminator
with open('D.pkl', 'wb') as f:
    pkl.dump(D, f)


# download files into the local machine
files.download('D.pkl')
files.download('G.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# from torch.autograd import Variable
# #device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
# bs = 100 #Batch Size
# z_dim = 100 #Z dimensions
# z = Variable(torch.randn(bs, z_dim).to(device))



sample_size=200
#create 100 latent vectors which has 100  dimension
z = np.random.uniform(-1, 1, size=(sample_size, z_size))
z = torch.from_numpy(z).float()

print(z[0].size())

G.eval() # eval mode

# set image directory root location
root = '/content'


for i, image_tensor in enumerate(z):
    image_tensor = image_tensor.detach()
    print(i,image_tensor)

    # save the latent vector, related to the image
    f = open(root + r'/images/%d.txt' %i,'a')
    f.write(np.array2string(image_tensor.numpy(), separator=','))
    f.close()

    # create image using latent vector
    rand_image = G(image_tensor.to(device)) 
    #view_samples(0, [rand_images])
    #print(rand_image.size())

    # save image
    rand_image = rand_image.view(-1, 28) # convert [784] tensor in to [28,28] tensor to create image
    print(rand_image.size())
    save_image(rand_image, root + r'/images/%d.png' %i, normalize = True)


#create a zip file contains generated image files and Latent vectors
shutil.make_archive('images', 'zip', 'images')

In [15]:
files.download('D.pkl')
files.download('G.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#unzip the zipped file
!unzip '/content/Fake_Digits.zip' -d '/content/Fake_Digits'

In [16]:
#open previously created Generator

with open('G.pkl', 'rb') as f:
    G = pkl.load(f)

In [ ]:
root = '/content'
print(len(train_dataset))
for idx, (img, _) in enumerate(train_dataset):
    # save the latent vector, related to the image
    print('1',idx,img.size())
    f = open(root + r'/real_images/%d.txt' %idx,'a')
    f.write(np.array2string(img.numpy(), separator=','))
    f.close()
    save_image(img, root + r'/real_images/%d.png' %idx, normalize = True)
    print('2',idx)
    if idx == 100:
        break   

#create a zip file contains generated image files and Latent vectors
shutil.make_archive('real_images', 'zip', 'real_images')
files.download('real_images.zip')
